<a href="https://colab.research.google.com/github/vladargunov/CreditDerivativesSimulator/blob/v11/templates/notebooks/template_notebook_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\Huge Version 1.1$

# Template for Developing Credit Trading Strategies

What's new in version 1.1?

- Added corrected Sharpe metric (see below how it is computed)

- Added [Maximum Drawdown](https://www.investopedia.com/terms/m/maximum-drawdown-mdd.asp) and [Annualised Return](https://www.investopedia.com/terms/a/annualized-total-return.asp) (over the whole period and over the last 252 days)

- Added new tickers (IHYG, HYG, LQD, IEAC, VIX)

- Find [templates](https://github.com/vladargunov/CreditDerivativesSimulator/tree/v11/templates/strategies) for strategies written by our team

- Now you can choose the project where you would like to submit your metrics, at this stage we have two of them: "Test" project used for experimenting with your strategies and "Final" for submitting your strategies for comparison and presentation. (Yet to implement)

- Get training data without running a simulator to perform your EDA

- Get testing iterator for debugging how your strategy performs in simulator





---

Let's start exploring the new features of the simulator!

Please set a version of simulator that you require, it is very advisable to use the latest version "v1.1".

In [1]:
VERSION =  "v1.1" #@param {type:"string"}


In [ ]:
#@title [RUN] Install required python libraries and clone the repo
# Assert for correct version

assert VERSION in ['v1.0', 'v1.1'], 'Specify correct version!' + \
                                'Available versions are "v1.0" and "v1.1"'

!pip install wandb -q
!git clone -b $VERSION https://github.com/vladargunov/CreditDerivativesSimulator.git

# Import necessary classses
%cd CreditDerivativesSimulator
from src.base_strategy import BaseStrategy
from src.simulator import Simulator

# Migration from previous version

All you need to do is to copy your strategy class from your previous notebook here! Or you can refer to the collection of our written [template strategies](https://github.com/vladargunov/CreditDerivativesSimulator/tree/v11/templates/strategies). If you do not know how to write your custom strategies, please refer to our previous template notebook.

In [3]:
class MyFirstStrategy(BaseStrategy):
  def train_model(self, train_data):
    pass

  def trade(self, daily_data) -> dict:
    return {'spx' : .1}

# Testing your strategy

In addition to previous features, note that you also need to define the "project_name" argument in simulator, which can be either "Test" or "Final", depending on what project you want to submit your strategy. Remember, use "Final" if you only want to submit your final version of the strategy.

You can also set "train_test_split_time" argument, which specifies the date from which you want your strategy to start trading. As a recommendation, for 'Test' runs you can set any desirable date, but for 'Final' projects let us agree about the unique date so it is easier to compare our strategies. For example, such date can be '2019-01-02'.

Please also do not forget to give your strategies meaningful names, which are defined in argument "run_name".

In [ ]:
# Create an instance of your strategy and of simulator
my_strategy = MyFirstStrategy()


# If you use wandb, you will need to past an API key from your wandb account
sim = Simulator(use_wandb=True, 
                debug_mode=False, 
                train_test_split_time='2019-01-02', # submit date in format 'yyyy-mm-dd'
                run_name='MyFirstStrategy1',
                project_name='Final') # or 'Final'

In [ ]:
# Available tickers in new version
sim.get_availiable_assets()

In [ ]:
# Simulate your strategy
sim.simulate(strategy=my_strategy)

# Debugging tools

Now you can access the training and testing data without running a strategy. Firstly initialise a simulator, and then call these functions (without calling 'sim.simulate'):

In [11]:
# Initialise a simulator for getting train and test data
debug_sim = Simulator(use_wandb=False,
                      train_test_split_time='2019-01-02')

In [ ]:
# Get training data
training_data = debug_sim.get_training_data()

training_data.head()

In [18]:
# Get test data at each trading day:

# Initialise an iterator
test_data = debug_sim.get_test_data()

In [ ]:
# Get what trade() function receives at each step
# Each time you call this cell, you will receive data for next
# trade day, to reset test data call previous cell
for trade_day in test_data:
  print(trade_day)
  break

# Details about the computation of Sharpe, Maximum Drawdown, and Annualised return

- Sharpe is calculated by the following formula:

$$ Sharpe = \frac{average \; daily \; return - risk \; free \;rate}{standard \; deviation \; daily \; return} $$

where risk free rate is computed as an average of 1 month treasury yields over the trading interval, which is then converted to an average daily rate by the formula

$$ risk \; free \;rate \; daily = (1 + risk \; free \;rate \; annual)^{1 / 365} - 1 $$

- Maximum Drawdown is computed by the formula:

$$ Maximum \; Drawdown = \frac{lowest \; value - highest \; value}{highest \; value} $$

where the highest and lowest values are specified over the trailing period.

- Annualised return is computed as:

$$ Annualised \; return = [(1 + daily \; return_1) * (1 + daily \; return_2) * ... * (1 + daily \; return_n) ]^{1 / n} - 1 $$

Where n is chosen according to the prespecified period.
